In [1]:
!pip install ultralytics==8.2.103 "opencv-python-headless<4.9" easyocr scikit-learn pandas numpy -q

from IPython import display
display.clear_output()

# Import libraries
import ultralytics
from ultralytics import YOLO
import os
import pandas as pd
import numpy as np
import random
import shutil
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from pathlib import Path
from sklearn.cluster import KMeans
import easyocr
from collections import defaultdict, Counter

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Prevent ultralytics from tracking
!yolo settings sync=False
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.13.7 torch-2.9.1 CPU (Apple M4)
Setup complete ✅ (10 CPUs, 16.0 GB RAM, 132.2/460.4 GB disk)


In [2]:
# --- Generic Helper Functions ---
def load_dataset_split(base_dir, split_name):
    """Loads a dataset split into a dataframe."""
    split_dir = Path(base_dir) / split_name
    images_dir = split_dir / 'images_orig'
    labels_dir = split_dir / 'labels_orig'

    # --- NEW CHECK ---
    if not images_dir.exists():
        raise FileNotFoundError(
            f"Error: The directory '{images_dir}' does not exist. "
            "Please check that your dataset downloaded and unzipped correctly. "
            f"Expected structure: {base_dir}/{split_name}/images"
        )
    # --- END NEW CHECK ---
    
    image_files = sorted(images_dir.glob('*.jpg'))

    # --- NEW CHECK ---
    if not image_files:
        raise FileNotFoundError(
            f"Error: No '.jpg' files were found in '{images_dir}'. "
            "The directory is empty or the download may have failed."
        )
    # --- END NEW CHECK ---
    
    data = []
    for img_path in image_files:
        label_filename = img_path.stem + '.txt'
        label_path = labels_dir / label_filename
        data.append({
            'image_path': str(img_path),
            'label_path': str(label_path) if label_path.exists() else None,
            'filename': img_path.name,
            'split': split_name
        })
    
    print(f"  Loaded {len(data)} images from {images_dir}") # Added for visibility
    return pd.DataFrame(data)

# --- Rotation/Augmentation Functions ---

def rotate_bbox_90(x_center, y_center, width, height, angle):
    """Rotate bounding box coordinates for 90, 180, or 270 degree rotations."""
    if angle == 90:
        return 1 - y_center, x_center, height, width
    elif angle == 180:
        return 1 - x_center, 1 - y_center, width, height
    elif angle == 270:
        return y_center, 1 - x_center, height, width
    raise ValueError("Angle must be 90, 180, or 270")

def rotate_keypoint_90(x, y, angle):
    """Rotate keypoint coordinates for 90, 180, or 270 degree rotations."""
    if angle == 90:
        return 1 - y, x
    elif angle == 180:
        return 1 - x, 1 - y
    elif angle == 270:
        return y, 1 - x
    raise ValueError("Angle must be 90, 180, or 270")

def rotate_image_90(image_path, angle):
    """Rotate an image by 90, 180, or 270 degrees."""
    img = Image.open(image_path)
    if angle == 90:
        return img.rotate(-90, expand=True)
    elif angle == 180:
        return img.rotate(-180, expand=True)
    elif angle == 270:
        return img.rotate(-270, expand=True)
    raise ValueError("Angle must be 90, 180, or 270")

# --- Field Dataset (Keypoints) Functions ---
FIELD_CLASS_NAMES = ['pitch']
NUM_KEYPOINTS = 32

def parse_field_label(label_path):
    """Parse a YOLO format label file with keypoints."""
    if label_path is None or not Path(label_path).exists():
        return []
    annotations = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                class_id, x_c, y_c, w, h = int(parts[0]), float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])
                keypoints = []
                num_visible_keypoints = 0
                for i in range(5, len(parts), 3):
                    if i + 2 < len(parts):
                        kpt_x, kpt_y, kpt_vis = float(parts[i]), float(parts[i+1]), int(float(parts[i+2]))
                        keypoints.append((kpt_x, kpt_y, kpt_vis))
                        if kpt_vis > 0: num_visible_keypoints += 1
                annotations.append({
                    'class_id': class_id, 'class_name': FIELD_CLASS_NAMES[class_id],
                    'x_center': x_c, 'y_center': y_c, 'width': w, 'height': h,
                    'keypoints': keypoints, 'num_visible_keypoints': num_visible_keypoints
                })
    return annotations

def add_field_label_info(df):
    """Add field label info to the dataframe."""
    df = df.copy()
    df['annotations'] = df['label_path'].apply(parse_field_label)
    df['num_pitches'] = df['annotations'].apply(len)
    df['avg_visible_keypoints'] = df['annotations'].apply(
        lambda anns: np.mean([ann['num_visible_keypoints'] for ann in anns]) if anns else 0
    )
    return df

def augment_field_dataset(df, base_dir, split_name, prob=0.5, angles=[90, 180, 270]):
    """Apply rotation augmentation to the field dataset."""
    aug_images_dir = Path(base_dir) / split_name / 'images_augmented'
    aug_labels_dir = Path(base_dir) / split_name / 'labels_augmented'
    aug_images_dir.mkdir(exist_ok=True); aug_labels_dir.mkdir(exist_ok=True)
    
    print(f"Augmenting field {split_name} set...")
    aug_count = 0
    for idx, row in df.iterrows():
        if random.random() < prob:
            angle = random.choice(angles)
            rotated_img = rotate_image_90(row['image_path'], angle)
            
            base_filename = Path(row['filename']).stem
            new_filename = f"{base_filename}_rot{angle}.jpg"
            new_img_path = aug_images_dir / new_filename
            new_label_path = aug_labels_dir / f"{base_filename}_rot{angle}.txt"
            rotated_img.save(new_img_path)
            
            with open(new_label_path, 'w') as f:
                for ann in row['annotations']:
                    new_x, new_y, new_w, new_h = rotate_bbox_90(
                        ann['x_center'], ann['y_center'], ann['width'], ann['height'], angle
                    )
                    
                    label_line = f"{ann['class_id']} {new_x} {new_y} {new_w} {new_h}"
                    for kpt_x, kpt_y, kpt_vis in ann['keypoints']:
                        new_kpt_x, new_kpt_y = (rotate_keypoint_90(kpt_x, kpt_y, angle) if kpt_vis > 0 else (kpt_x, kpt_y))
                        label_line += f" {new_kpt_x} {new_kpt_y} {kpt_vis}"
                    f.write(label_line + "\n")
            aug_count += 1
    print(f"  Created {aug_count} augmented field samples for {split_name}.")

# --- Player Dataset (Detection) Functions ---
PLAYER_CLASS_NAMES = ['ball', 'goalkeeper', 'player', 'referee']

def parse_player_label(label_path):
    """Parse a YOLO format label file (detection)."""
    if label_path is None or not Path(label_path).exists():
        return []
    annotations = []
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                class_id, x_c, y_c, w, h = int(parts[0]), float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])
                annotations.append({
                    'class_id': class_id, 'class_name': PLAYER_CLASS_NAMES[class_id],
                    'x_center': x_c, 'y_center': y_c, 'width': w, 'height': h
                })
    return annotations

def add_player_label_info(df):
    """Add player label info to the dataframe."""
    df = df.copy()
    df['annotations'] = df['label_path'].apply(parse_player_label)
    df['num_objects'] = df['annotations'].apply(len)
    for class_id, class_name in enumerate(PLAYER_CLASS_NAMES):
        df[f'num_{class_name}s'] = df['annotations'].apply(
            lambda anns: sum(1 for ann in anns if ann['class_id'] == class_id)
        )
    return df

def augment_player_dataset(df, base_dir, split_name, prob=0.5, angles=[90, 180, 270]):
    """Apply rotation augmentation to the player dataset."""
    aug_images_dir = Path(base_dir) / split_name / 'images_augmented'
    aug_labels_dir = Path(base_dir) / split_name / 'labels_augmented'
    aug_images_dir.mkdir(exist_ok=True); aug_labels_dir.mkdir(exist_ok=True)
    
    print(f"Augmenting player {split_name} set...")
    aug_count = 0
    for idx, row in df.iterrows():
        if random.random() < prob:
            angle = random.choice(angles)
            rotated_img = rotate_image_90(row['image_path'], angle)
            
            base_filename = Path(row['filename']).stem
            new_filename = f"{base_filename}_rot{angle}.jpg"
            new_img_path = aug_images_dir / new_filename
            new_label_path = aug_labels_dir / f"{base_filename}_rot{angle}.txt"
            rotated_img.save(new_img_path)
            
            with open(new_label_path, 'w') as f:
                for ann in row['annotations']:
                    new_x, new_y, new_w, new_h = rotate_bbox_90(
                        ann['x_center'], ann['y_center'], ann['width'], ann['height'], angle
                    )
                    f.write(f"{ann['class_id']} {new_x} {new_y} {new_w} {new_h}\n")
            aug_count += 1
    print(f"  Created {aug_count} augmented player samples for {split_name}.")

# ...existing code...
from pathlib import Path
import shutil

def copy_originals_to_yolo(data_dir):
    data_dir = Path(data_dir)
    for split in ['train','valid','test']:
        src_images = data_dir / split / 'images_orig'
        src_labels = data_dir / split / 'labels_orig'
        dst_images = data_dir / split / 'images'
        dst_labels = data_dir / split / 'labels'
        dst_images.mkdir(parents=True, exist_ok=True)
        dst_labels.mkdir(parents=True, exist_ok=True)
        if src_images.exists():
            for f in src_images.glob('*.jpg'):
                shutil.copy(f, dst_images)
        if src_labels.exists():
            for f in src_labels.glob('*.txt'):
                shutil.copy(f, dst_labels)
    print(f"Copied originals for {data_dir}")
# ...existing code...
# --- YAML Creation Function ---
def create_dataset_yaml(data_dir, class_names, kpt_shape=None, use_augmentation=True):
    """Creates the data.yaml file for YOLO training."""
    data_dir = Path(data_dir)
    
    # Create main directories if they don't exist
    for split in ['train', 'valid', 'test']:
        (data_dir / split).mkdir(parents=True, exist_ok=True)
        (data_dir / split / 'images').mkdir(parents=True, exist_ok=True)
        (data_dir / split / 'labels').mkdir(parents=True, exist_ok=True)

    
    
    # Move original images/labels
    # shutil.move(str(data_dir / 'train' / 'images'), str(data_dir / 'train' / 'images_orig'))
    # shutil.move(str(data_dir / 'train' / 'labels'), str(data_dir / 'train' / 'labels_orig'))
    # shutil.move(str(data_dir / 'valid' / 'images'), str(data_dir / 'valid' / 'images_orig'))
    # shutil.move(str(data_dir / 'valid' / 'labels'), str(data_dir / 'valid' / 'labels_orig'))
    # shutil.move(str(data_dir / 'test' / 'images'), str(data_dir / 'test' / 'images_orig'))
    # shutil.move(str(data_dir / 'test' / 'labels'), str(data_dir / 'test' / 'labels_orig'))
    
    # # Rename augmented dirs to be the new train/valid/test dirs
    # shutil.move(str(data_dir / 'train' / 'images_augmented'), str(data_dir / 'train' / 'images'))
    # shutil.move(str(data_dir / 'train' / 'labels_augmented'), str(data_dir / 'train' / 'labels'))
    # shutil.move(str(data_dir / 'valid' / 'images_augmented'), str(data_dir / 'valid' / 'images'))
    # shutil.move(str(data_dir / 'valid' / 'labels_augmented'), str(data_dir / 'valid' / 'labels'))
    # shutil.move(str(data_dir / 'test' / 'images_augmented'), str(data_dir / 'test' / 'images'))
    # shutil.move(str(data_dir / 'test' / 'labels_augmented'), str(data_dir / 'test' / 'labels'))
    
    if use_augmentation:
        for split in ['train', 'valid', 'test']:
            aug_images = data_dir / split / 'images_augmented'
            aug_labels = data_dir / split / 'labels_augmented'
            if aug_images.exists():
                for f in aug_images.glob('*.jpg'):
                    shutil.copy(f, data_dir / split / 'images')
            if aug_labels.exists():
                for f in aug_labels.glob('*.txt'):
                    shutil.copy(f, data_dir / split / 'labels')
    
    # Create data.yaml

    train_abs_path = (data_dir / 'train' / 'images').absolute()
    val_abs_path = (data_dir / 'valid' / 'images').absolute()
    test_abs_path = (data_dir / 'test' / 'images').absolute()
    
    # Create data.yaml
    yaml_content = f"""
    train: {train_abs_path}
    val: {val_abs_path}
    test: {test_abs_path}

    names: {class_names}
    nc: {len(class_names)}
    """
    
    if kpt_shape:
        yaml_content += f"\nkpt_shape: {kpt_shape}\n"
        
    yaml_path = data_dir / 'data.yaml'
    with open(yaml_path, 'w') as f:
        f.write(yaml_content)
        
    print(f"Created {yaml_path} with ABSOLUTE paths.")
    return str(yaml_path)

In [3]:
# --- Process Player Dataset ---
player_base_dir = 'player_dataset'
player_train_df = load_dataset_split(player_base_dir, 'train')
player_valid_df = load_dataset_split(player_base_dir, 'valid')
player_test_df = load_dataset_split(player_base_dir, 'test')

player_train_df = add_player_label_info(player_train_df)
player_valid_df = add_player_label_info(player_valid_df)
player_test_df = add_player_label_info(player_test_df)

augment_player_dataset(player_train_df, player_base_dir, 'train', prob=0.5)
augment_player_dataset(player_valid_df, player_base_dir, 'valid', prob=0.3)
augment_player_dataset(player_test_df, player_base_dir, 'test', prob=0.3)

copy_originals_to_yolo('player_dataset')
player_yaml_path = create_dataset_yaml(player_base_dir, PLAYER_CLASS_NAMES)

# --- Process Field Dataset ---
field_base_dir = 'field_dataset'
field_train_df = load_dataset_split(field_base_dir, 'train')
field_valid_df = load_dataset_split(field_base_dir, 'valid')
field_test_df = load_dataset_split(field_base_dir, 'test')

field_train_df = add_field_label_info(field_train_df)
field_valid_df = add_field_label_info(field_valid_df)
field_test_df = add_field_label_info(field_test_df)

augment_field_dataset(field_train_df, field_base_dir, 'train', prob=0.5)
augment_field_dataset(field_valid_df, field_base_dir, 'valid', prob=0.3)
augment_field_dataset(field_test_df, field_base_dir, 'test', prob=0.3)

copy_originals_to_yolo('field_dataset')
field_yaml_path = create_dataset_yaml(field_base_dir, FIELD_CLASS_NAMES, kpt_shape=[NUM_KEYPOINTS, 3])

  Loaded 298 images from player_dataset/train/images_orig
  Loaded 49 images from player_dataset/valid/images_orig
  Loaded 25 images from player_dataset/test/images_orig
Augmenting player train set...
  Created 148 augmented player samples for train.
Augmenting player valid set...
  Created 10 augmented player samples for valid.
Augmenting player test set...
  Created 12 augmented player samples for test.
Copied originals for player_dataset
Created player_dataset/data.yaml with ABSOLUTE paths.
  Loaded 255 images from field_dataset/train/images_orig
  Loaded 34 images from field_dataset/valid/images_orig
  Loaded 28 images from field_dataset/test/images_orig
Augmenting field train set...
  Created 129 augmented field samples for train.
Augmenting field valid set...
  Created 12 augmented field samples for valid.
Augmenting field test set...
  Created 9 augmented field samples for test.
Copied originals for field_dataset
Created field_dataset/data.yaml with ABSOLUTE paths.


In [4]:
# ...existing code...
from pathlib import Path

def verify_dataset_for_training(data_dir):
    data_dir = Path(data_dir)
    for split in ['train','valid','test']:
        imgs = list((data_dir / split / 'images').glob('*.jpg'))
        lbls = list((data_dir / split / 'labels').glob('*.txt'))
        aug_imgs = list((data_dir / split / 'images_augmented').glob('*.jpg')) if (data_dir / split / 'images_augmented').exists() else []
        aug_lbls = list((data_dir / split / 'labels_augmented').glob('*.txt')) if (data_dir / split / 'labels_augmented').exists() else []
        print(f"{data_dir}/{split}: images={len(imgs)}, labels={len(lbls)}, aug_images={len(aug_imgs)}, aug_labels={len(aug_lbls)}")
        # Print sample mismatches
        img_stems = {p.stem for p in imgs}
        lbl_stems = {p.stem for p in lbls}
        missing_labels = sorted(list(img_stems - lbl_stems))[:5]
        if missing_labels:
            print(f"  WARNING: {len(missing_labels)} images missing labels (examples): {missing_labels}")
        missing_images = sorted(list(lbl_stems - img_stems))[:5]
        if missing_images:
            print(f"  WARNING: {len(missing_images)} labels without images (examples): {missing_images}")
    print("Verification done.")

# Run verification for both datasets
verify_dataset_for_training('player_dataset')
verify_dataset_for_training('field_dataset')
# ...existing code...

player_dataset/train: images=821, labels=821, aug_images=365, aug_labels=365
player_dataset/valid: images=102, labels=102, aug_images=40, aug_labels=40
player_dataset/test: images=57, labels=57, aug_images=22, aug_labels=22
Verification done.
field_dataset/train: images=597, labels=597, aug_images=342, aug_labels=342
field_dataset/valid: images=64, labels=64, aug_images=30, aug_labels=30
field_dataset/test: images=55, labels=55, aug_images=27, aug_labels=27
Verification done.


In [5]:
import ultralytics
from ultralytics import YOLO

# --- Configuration for Maximum Stability ---
# Lowest resource footprint settings for CPU training
STABLE_BATCH_SIZE = 4 
STABLE_WORKERS = 0  
LIGHTWEIGHT_IMG_SIZE = 320 # <--- NEW: Use 320x320 pixels instead of 640x640

print(f"Using LIGHTWEIGHT Settings: Batch={STABLE_BATCH_SIZE}, Workers={STABLE_WORKERS}, Image Size={LIGHTWEIGHT_IMG_SIZE}")

print("\n--- Starting Player Detector Training (Lightweight Mode) ---")

player_model = YOLO('yolov8n.pt') 

player_detector_results = player_model.train(
    data=player_yaml_path,
    epochs=10,            
    imgsz=LIGHTWEIGHT_IMG_SIZE, # <--- CRITICAL CHANGE
    project='Initial_Evaluation',
    name='player_detection_lightweight',
    batch=STABLE_BATCH_SIZE, 
    workers=STABLE_WORKERS   
)
print("Player Detector Training Complete.")

Using LIGHTWEIGHT Settings: Batch=4, Workers=0, Image Size=320

--- Starting Player Detector Training (Lightweight Mode) ---
New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.13.7 torch-2.9.1 CPU (Apple M4)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=player_dataset/data.yaml, epochs=10, time=None, patience=100, batch=4, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=0, project=Initial_Evaluation, name=player_detection_lightweight6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride

train: Scanning /Users/adity/Downloads/CS_4644/FIFA_player_detection/player_dataset/train/labels... 821 images, 0 backgrounds, 0 corrupt: 100%|██████████| 821/821 [00:00<00:00, 2382.86it/s]

train: New cache created: /Users/adity/Downloads/CS_4644/FIFA_player_detection/player_dataset/train/labels.cache



/Users/adity/Downloads/CS_4644/FIFA_player_detection/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning /Users/adity/Downloads/CS_4644/FIFA_player_detection/player_dataset/valid/labels... 102 images, 0 backgrounds, 0 corrupt: 100%|██████████| 102/102 [00:00<00:00, 2929.50it/s]

val: New cache created: /Users/adity/Downloads/CS_4644/FIFA_player_detection/player_dataset/valid/labels.cache


...

      10/10         0G      2.096      1.086     0.8041         24        320: 100%|██████████| 206/206 [01:10<00:00,  2.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  4.64it/s]

                   all        102       2447      0.596      0.243      0.246     0.0894



10 epochs completed in 0.195 hours.
Optimizer stripped from Initial_Evaluation/player_detection_lightweight6/weights/last.pt, 6.2MB
Optimizer stripped from Initial_Evaluation/player_detection_lightweight6/weights/best.pt, 6.2MB

Validating Initial_Evaluation/player_detection_lightweight6/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.13.7 torch-2.9.1 CPU (Apple M4)
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:02<00:00,  4.43it/s]


                   all        102       2447      0.594      0.243      0.246     0.0894
                  ball         96         96          1          0          0          0
            goalkeeper         80         81      0.475      0.112      0.178     0.0553
                player        102       2029      0.648      0.631      0.629      0.228
               referee        102        241      0.253      0.228      0.178     0.0741
Speed: 0.1ms preprocess, 12.0ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to Initial_Evaluation/player_detection_lightweight6
Player Detector Training Complete.


In [7]:
# ...existing code...
import time
from pathlib import Path
from ultralytics import YOLO

# keep seeds / config as before
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
import torch
torch.manual_seed(SEED)

EPOCHS = 10
IMG_SZ = 320
BATCH = 4
WORKERS = 0

def train_and_eval_random(run_name_prefix='random_scratch'):
    run_name = f"{run_name_prefix}_{int(time.time())}"  # unique name avoids auto-increment confusion
    print(f"\n--- Training random-init run: {run_name} ---")
    model = YOLO('yolov8n.yaml')  # <-- randomized initialization
    model.train(
        data=player_yaml_path,
        epochs=EPOCHS,
        imgsz=IMG_SZ,
        batch=BATCH,
        workers=WORKERS,
        project='initial_eval_compare',
        name=run_name,
        verbose=False,
    )

    project_dir = Path('initial_eval_compare')
    # find the latest directory that starts with the run name prefix
    candidates = [d for d in project_dir.iterdir() if d.is_dir() and d.name.startswith(run_name_prefix)]
    run_dir = sorted(candidates, key=lambda p: p.stat().st_mtime, reverse=True)[0]
    print(f"Using run directory: {run_dir}")

    best_path = run_dir / 'weights' / 'best.pt'
    if not best_path.exists():
        best_path = run_dir / 'weights' / 'last.pt'
    if not best_path.exists():
        raise FileNotFoundError(f"No weights found in {run_dir}/weights")

    print(f"Evaluating {best_path} ...")
    val_res = YOLO(str(best_path)).val(data=player_yaml_path)
    print("Done. Validation results printed above.")
    return val_res

# Run only the randomized-weights experiment
res_scratch = train_and_eval_random('random_scratch')
# ...existing code...


--- Training random-init run: random_scratch_1764540565 ---
New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.13.7 torch-2.9.1 CPU (Apple M4)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=player_dataset/data.yaml, epochs=10, time=None, patience=100, batch=4, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=0, project=initial_eval_compare, name=random_scratch_1764540565, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnost

train: Scanning /Users/adity/Downloads/CS_4644/FIFA_player_detection/player_dataset/train/labels.cache... 821 images, 0 backgrounds, 0 corrupt: 100%|██████████| 821/821 [00:00<?, ?it/s]
/Users/adity/Downloads/CS_4644/FIFA_player_detection/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning /Users/adity/Downloads/CS_4644/FIFA_player_detection/player_dataset/valid/labels.cache... 102 images, 0 backgrounds, 0 corrupt: 100%|██████████| 102/102 [00:00<?, ?it/s]

Plotting labels to initial_eval_compare/random_scratch_1764540565/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to initial_eval_compare/random_scratch_1764540565
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


/Users/adity/Downloads/CS_4644/FIFA_player_detection/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
       1/10         0G  0.0002544      6.439  0.0001426         22        320: 100%|██████████| 206/206 [01:22<00:00,  2.49it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:04<00:00,  3.04it/s]

...


10 epochs completed in 0.382 hours.
Optimizer stripped from initial_eval_compare/random_scratch_1764540565/weights/last.pt, 6.2MB
Optimizer stripped from initial_eval_compare/random_scratch_1764540565/weights/best.pt, 6.2MB

Validating initial_eval_compare/random_scratch_1764540565/weights/best.pt...
Ultralytics YOLOv8.2.103 🚀 Python-3.13.7 torch-2.9.1 CPU (Apple M4)
YOLOv8n summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:06<00:00,  1.90it/s]


                   all        102       2447       0.86     0.0579     0.0508     0.0126
Speed: 0.5ms preprocess, 38.7ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to initial_eval_compare/random_scratch_1764540565
Using run directory: initial_eval_compare/random_scratch_1764540565
Evaluating initial_eval_compare/random_scratch_1764540565/weights/best.pt ...
Ultralytics YOLOv8.2.103 🚀 Python-3.13.7 torch-2.9.1 CPU (Apple M4)
YOLOv8n summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /Users/adity/Downloads/CS_4644/FIFA_player_detection/player_dataset/valid/labels.cache... 102 images, 0 backgrounds, 0 corrupt: 100%|██████████| 102/102 [00:00<?, ?it/s]
/Users/adity/Downloads/CS_4644/FIFA_player_detection/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:10<00:00,  1.55s/it]


                   all        102       2447      0.859      0.058     0.0508     0.0127
                  ball         96         96          1          0          0          0
            goalkeeper         80         81          1          0          0          0
                player        102       2029      0.437      0.232      0.197     0.0491
               referee        102        241          1          0    0.00656    0.00152
Speed: 0.5ms preprocess, 71.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /Users/adity/Downloads/CS_4644/FIFA_player_detection/runs/detect/val
Done. Validation results printed above.
